In [54]:
import pprint
import requests
import sys
import pandas as pd
from datetime import datetime

api_key = "9b656c6f5237727cdf1e35288a7afa8c"
geo_URL = 'http://api.openweathermap.org/geo/1.0/direct'

loc = [
  'Buenos Aires, Argentina',
  'Guangzhou, China',
  'Wichita, Kansas',
  'Niskayuna, New York',
  'Gwangmyeong, South Korea',
  'Taipei, Taiwan',
  'Nanaimo, British Columbia',
  'Chennai, India',
  'Barrington, Illinois',
  'Littleton, Colorado',
  'Peterhead, Scotland',
  'Vizag, India',
  'Des Moines, Iowa',
  'Beijing, China',
  'Killeen, Texas',
  'Morehead City, North Carolina'
]

df_wide = pd.DataFrame(index=loc)
######

for city in loc:
    geo = f'{geo_URL}?q={city}&limit=5&appid={api_key}'
    resp = requests.get( geo )

    if resp.status_code != 200: # Failure?
        print( f'Error geocoding {city}: {resp.status_code}' )
        sys.exit( 1 )

    # OpenWeatherMap returns a list of matching cities, up to the limit specified
    # in the API call; even if you only ask for one city (limit=5), it's still
    # returned as a 1-element list

    if len( resp.json() ) == 0: # No such city?
        print( f'Error locating city {city}; {resp.status_code}' )
        sys.exit( 2 )

    json = resp.json()
    if type( json ) == list: # List of cities?
        lat = json[ 0 ][ 'lat' ]
        lon = json[ 0 ][ 'lon' ]
    else: # Unknown city?
        print( f'Error, invalid data returned for city {city}, {resp.status_code}' )
        sys.exit( 3 )

    # Use Peterhead's latitude and longitude to get its 5-day forecast in 3-hour blocks

    forecast_URL = 'http://api.openweathermap.org/data/2.5/forecast'
    forecast = f'{forecast_URL}?lat={lat}&lon={lon}&appid={api_key}'
    resp = requests.get( forecast )

    if resp.status_code != 200: # Failure?
        print( f'Error retrieving data: {resp.status_code}' )
        sys.exit( 4 )

    data = resp.json()
##### 

    data_list = []
    for i in range(len(data['list'])):
        forecast_data = data[ 'list' ][ i ]
        dt = datetime.strptime(forecast_data['dt_txt'], '%Y-%m-%d %H:%M:%S')
        temp_max = forecast_data['main']['temp_max']
        temp_min = forecast_data['main']['temp_min']
        data_list.append({
            'city': city,
            'datetime': dt,
            'temp_max': temp_max,
            'temp_min': temp_min
        })

    df = pd.DataFrame(data_list)
#####

    df_grouped = df.groupby(pd.Grouper(key='datetime', axis=0, freq='D')).aggregate({
        'city': lambda x: x.iloc[0],
        'temp_max': 'max',
        'temp_min': 'min'})
   
    df_grouped['min_avg'] = df_grouped['temp_min'].mean()
    df_grouped['max_avg'] = df_grouped['temp_max'].mean()
#####

    for i, day in enumerate(df_grouped.index, start=1):
        if i>4:
            break
        df_wide.loc[city, f'Min_{i}'] = df_grouped.loc[day, 'temp_min']
        df_wide.loc[city, f'Max_{i}'] = df_grouped.loc[day, 'temp_max']



df_wide = df_wide.reset_index()
df_wide = df_wide.rename({'index': 'City'}, axis='columns')
df_wide['Min_Avg'] = df_wide[['Min_1', 'Min_2', 'Min_3', 'Min_4']].mean(axis=1).round(2)
df_wide['Max_Avg'] = df_wide[['Max_1', 'Max_2', 'Max_3', 'Max_4']].mean(axis=1).round(2)
df_wide

,City,Min_1,Max_1,Min_2,Max_2,Min_3,Max_3,Min_4,Max_4,Min_Avg,Max_Avg
0,"Buenos Aires, Argentina",276.55,280.75,277.08,280.74,278.40,282.87,278.53,283.17,277.64,281.88
1,"Guangzhou, China",301.86,309.72,301.92,307.00,302.19,306.90,302.26,307.77,302.06,307.85
2,"Wichita, Kansas",293.99,306.36,295.15,308.42,297.49,312.54,299.63,311.98,296.56,309.82
3,"Niskayuna, New York",292.29,307.84,294.76,301.17,295.66,299.45,289.77,301.86,293.12,302.58
4,"Gwangmyeong, South Korea",295.20,298.59,295.59,302.52,295.82,302.67,297.09,304.21,295.92,302.00
5,"Taipei, Taiwan",300.34,308.38,300.22,308.68,300.93,309.01,301.05,309.79,300.64,308.96
6,"Nanaimo, British Columbia",291.63,297.28,289.73,298.33,288.08,295.88,287.66,294.06,289.28,296.39
7,"Chennai, India",301.93,304.64,301.94,307.84,302.79,306.79,302.39,307.49,302.26,306.69
8,"Barrington, Illinois",291.77,297.57,290.45,299.42,290.64,299.21,290.62,302.57,290.87,299.69
9,"Littleton, Colorado",289.77,304.45,289.81,307.02,292.51,309.21,294.38,312.56,291.62,308.31


In [58]:
df_wide.to_csv('temp.csv', index=False, float_format='%.2f')
